In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.metrics import matthews_corrcoef
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

/Users/Awesomekids/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/Awesomekids/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
2024-08-15 16:44:34.594287: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load data
rna = pd.read_csv('data/Tab_delimited_text/Hackathon2024.RNA.txt.gz', sep='\t', compression='gzip')
atac = pd.read_csv('data/Tab_delimited_text/Hackathon2024.ATAC.txt.gz', sep='\t', compression='gzip')
meta = pd.read_csv('data/Tab_delimited_text/Hackathon2024.Meta.txt.gz', sep='\t', compression='gzip')
train_pairs = pd.read_csv('data/Tab_delimited_text/Hackathon2024.Training.Set.Peak2Gene.Pairs.txt.gz', sep='\t', compression='gzip')
test_pairs = pd.read_csv('data/Tab_delimited_text/Hackathon2024.Testing.Set.Peak2Gene.Pairs.txt.gz', sep='\t', compression='gzip')


In [3]:
rna.head(20)

,gene,AAACCAACACAATGCC.1,AAACCAACAGGAACTG.1,AAACCAACATAATCCG.1,AAACCAACATTGTGCA.1,AAACCGCGTACTTCAC.1,AAACCGGCATAATCAC.1,AAACGCGCAGCAAGAT.1,AAACGGATCCCATAGG.1,AAAGCAAGTGCTAGCG.1,...,TTTGTCCCAGTAGGAT.1,TTTGTCTAGCTATTAG.1,TTTGTCTAGGACCTGC.1,TTTGTGAAGCGATACT.1,TTTGTGAAGGAACGGT.1,TTTGTGGCAGCAACCT.1,TTTGTGTTCATTGACA.1,TTTGTGTTCGTCAAGT.1,TTTGTGTTCTCCATAT.1,TTTGTTGGTCAGGAAG.1
0,MIR1302-2HG,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,FAM138A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,OR4F5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,AL627309.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,AL627309.3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,AL627309.2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,AL627309.5,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
7,AL627309.4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,AP006222.2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,AL732372.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
atac.head()

,peak,AAACCAACACAATGCC.1,AAACCAACAGGAACTG.1,AAACCAACATAATCCG.1,AAACCAACATTGTGCA.1,AAACCGCGTACTTCAC.1,AAACCGGCATAATCAC.1,AAACGCGCAGCAAGAT.1,AAACGGATCCCATAGG.1,AAAGCAAGTGCTAGCG.1,...,TTTGTCCCAGTAGGAT.1,TTTGTCTAGCTATTAG.1,TTTGTCTAGGACCTGC.1,TTTGTGAAGCGATACT.1,TTTGTGAAGGAACGGT.1,TTTGTGGCAGCAACCT.1,TTTGTGTTCATTGACA.1,TTTGTGTTCGTCAAGT.1,TTTGTGTTCTCCATAT.1,TTTGTTGGTCAGGAAG.1
0,chr1-10109-10357,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,chr1-180730-181630,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
2,chr1-191491-191736,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,chr1-267816-268196,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,chr1-586028-586373,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
meta.head()

,nCount_RNA,nCount_ATAC,CellType
AAACCAACACAATGCC-1,5849,16550,CD14 Mono
AAACCAACAGGAACTG-1,5901,25593,CD14 Mono
AAACCAACATAATCCG-1,7975,42743,CD14 Mono
AAACCAACATTGTGCA-1,5525,21760,CD14 Mono
AAACCGCGTACTTCAC-1,10327,76652,CD14 Mono


In [6]:
train_pairs.head()

,peak,gene,Pair,Peak2Gene
0,chr1-89196985-89201657,GBP2,chr1-89196985-89201657_GBP2,True
1,chr6-33077557-33083333,HLA-DPA1,chr6-33077557-33083333_HLA-DPA1,True
2,chr6-137789753-137792920,TNFAIP3,chr6-137789753-137792920_TNFAIP3,True
3,chr1-212604203-212626574,ATF3,chr1-212604203-212626574_ATF3,True
4,chr2-96541661-96555628,ARID5A,chr2-96541661-96555628_ARID5A,True


In [10]:
# Preprocess data
scaler = StandardScaler()

# Extract and align RNA and ATAC features for training pairs
train_rna = rna.set_index('gene').loc[train_pairs['gene']].values
train_atac = atac.set_index('peak').loc[train_pairs['peak']].values


In [11]:
# Scale RNA and ATAC features
train_rna_scaled = scaler.fit_transform(train_rna)
train_atac_scaled = scaler.fit_transform(train_atac)

# Combine RNA and ATAC features
X_train = np.hstack([train_rna_scaled, train_atac_scaled])
y_train = train_pairs['Peak2Gene'].map({True: 1, False: 0}).values


In [12]:
# Ensure consistent shapes
assert X_train.shape[0] == y_train.shape[0], "Mismatched number of samples in X_train and y_train"

# Balance the training data using SMOTE
smote = SMOTE()
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Build a neural network model
model = Sequential([
    Dense(128, input_dim=X_train_balanced.shape[1], activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

2024-08-15 16:49:51.293642: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_balanced, y_train_balanced, epochs=50, batch_size=32, validation_split=0.2)

# Prepare testing data
test_rna = rna.set_index('gene').loc[test_pairs['gene']].values
test_atac = atac.set_index('peak').loc[test_pairs['peak']].values

Epoch 1/50
8/8 [==============================] - 1s 22ms/step - loss: 0.8154 - accuracy: 0.6750 - val_loss: 0.9844 - val_accuracy: 0.8000
Epoch 2/50
8/8 [==============================] - 0s 6ms/step - loss: 1.2444 - accuracy: 0.6417 - val_loss: 1.6724 - val_accuracy: 0.7667
Epoch 3/50
8/8 [==============================] - 0s 7ms/step - loss: 1.1578 - accuracy: 0.7042 - val_loss: 1.3320 - val_accuracy: 0.8333
Epoch 4/50
8/8 [==============================] - 0s 6ms/step - loss: 0.9633 - accuracy: 0.7333 - val_loss: 1.8172 - val_accuracy: 0.7333
Epoch 5/50
8/8 [==============================] - 0s 7ms/step - loss: 0.7528 - accuracy: 0.8083 - val_loss: 1.6869 - val_accuracy: 0.7500
Epoch 6/50
8/8 [==============================] - 0s 6ms/step - loss: 0.6144 - accuracy: 0.7792 - val_loss: 1.3438 - val_accuracy: 0.8167
Epoch 7/50
8/8 [==============================] - 0s 6ms/step - loss: 0.6478 - accuracy: 0.7917 - val_loss: 1.4018 - val_accuracy: 0.8000
Epoch 8/50
8/8 [=================

In [14]:
test_rna_scaled = scaler.transform(test_rna)
test_atac_scaled = scaler.transform(test_atac)

X_test = np.hstack([test_rna_scaled, test_atac_scaled])

# Generate predictions
predictions = model.predict(X_test).flatten()
predictions = (predictions > 0.5).astype(int)

10/10 [==============================] - 0s 2ms/step


In [15]:
# Map predictions back to TRUE/FALSE
predicted_labels = pd.Series(predictions).map({1: 'TRUE', 0: 'FALSE'})

# Prepare submission
submission = test_pairs.copy()
submission['Peak2Gene'] = predicted_labels

# Save the submission file
submission.to_csv('prediction/prediction.csv', index=False)

In [21]:
from sklearn.metrics import matthews_corrcoef

# Generate predictions on the training set
train_predictions = model.predict(X_train_balanced).flatten()
train_predictions = (train_predictions > 0.5).astype(int)

# Calculate MCC for the training set
mcc_train = matthews_corrcoef(y_true=y_train_balanced, y_pred=train_predictions)
print(f'Matthews Correlation Coefficient (MCC) on Training Set: {mcc_train}')


10/10 [==============================] - 0s 1ms/step
Matthews Correlation Coefficient (MCC) on Training Set: 0.9354143466934853


In [4]:
# Step 2: Ensure all data in RNA and ATAC matrices are numeric
rna = rna.apply(pd.to_numeric, errors='coerce')
atac = atac.apply(pd.to_numeric, errors='coerce')

In [5]:
# Fill any NaN values with zeros (as these represent missing data)
rna.fillna(0, inplace=True)
atac.fillna(0, inplace=True)

In [6]:
# Step 3: Normalize RNA and ATAC data (only numeric data)
scaler = StandardScaler()
rna_scaled = scaler.fit_transform(rna.T).T  # Transpose to scale across samples and then transpose back
atac_scaled = scaler.fit_transform(atac.T).T

In [7]:
# Convert scaled data back to DataFrame for easier manipulation
rna_scaled_df = pd.DataFrame(rna_scaled, index=rna.index, columns=rna.columns)
atac_scaled_df = pd.DataFrame(atac_scaled, index=atac.index, columns=atac.columns)


In [8]:
# Step 4: Feature Selection and Data Preparation
def get_feature_vector(pair, rna_data, atac_data):
    gene = pair['gene']
    peak = pair['peak']
    
    gene_exp = rna_data.loc[gene].values if gene in rna_data.index else np.zeros(rna_data.shape[1])
    peak_acc = atac_data.loc[peak].values if peak in atac_data.index else np.zeros(atac_data.shape[1])
    
    # Additional features: interaction terms
    interaction = gene_exp * peak_acc
    
    return np.concatenate([gene_exp, peak_acc, interaction])

In [9]:
# Prepare training data
X_train = np.array([get_feature_vector(row, rna_scaled_df, atac_scaled_df) for idx, row in train_pairs.iterrows()])
y_train = train_pairs['Peak2Gene'].apply(lambda x: 1 if x == 'TRUE' else 0).values


In [15]:
from sklearn.utils import resample

# Check class distribution
class_counts = np.bincount(y_train)
print(f"Class distribution: {class_counts}")

# If only one class, just use existing data
if len(np.unique(y_train)) == 1:
    print("Only one class present. Using existing data without resampling.")
    X_train_balanced = X_train
    y_train_balanced = y_train
else:
    # Apply SMOTE if both classes are present
    smote = SMOTE(random_state=42)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)


Class distribution: [300]
Only one class present. Using existing data without resampling.


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Define the Keras model for binary classification
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_balanced.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Single output for binary classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Convert labels to binary format
y_train_balanced_cat = np.array(y_train_balanced)

In [17]:
# Step 7: Train the model
model.fit(X_train_balanced, y_train_balanced_cat, epochs=50, batch_size=32, validation_split=0.2)


Epoch 1/50
8/8 [==============================] - 1s 22ms/step - loss: 0.6915 - accuracy: 1.0000 - val_loss: 0.6892 - val_accuracy: 1.0000
Epoch 2/50
8/8 [==============================] - 0s 6ms/step - loss: 0.6875 - accuracy: 1.0000 - val_loss: 0.6852 - val_accuracy: 1.0000
Epoch 3/50
8/8 [==============================] - 0s 6ms/step - loss: 0.6836 - accuracy: 1.0000 - val_loss: 0.6812 - val_accuracy: 1.0000
Epoch 4/50
8/8 [==============================] - 0s 7ms/step - loss: 0.6796 - accuracy: 1.0000 - val_loss: 0.6773 - val_accuracy: 1.0000
Epoch 5/50
8/8 [==============================] - 0s 7ms/step - loss: 0.6757 - accuracy: 1.0000 - val_loss: 0.6734 - val_accuracy: 1.0000
Epoch 6/50
8/8 [==============================] - 0s 7ms/step - loss: 0.6718 - accuracy: 1.0000 - val_loss: 0.6695 - val_accuracy: 1.0000
Epoch 7/50
8/8 [==============================] - 0s 8ms/step - loss: 0.6679 - accuracy: 1.0000 - val_loss: 0.6656 - val_accuracy: 1.0000
Epoch 8/50
8/8 [=================

In [18]:
# Step 8: Prediction on Testing Data
X_test = np.array([get_feature_vector(row, rna_scaled_df, atac_scaled_df) for idx, row in test_pairs.iterrows()])
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)
y_pred_labels = ['TRUE' if y == 1 else 'FALSE' for y in y_pred]


10/10 [==============================] - 0s 998us/step


In [19]:
# Step 9: Write predictions to a new file
test_pairs['Peak2Gene'] = y_pred_labels
test_pairs.to_csv('prediction_keras.csv', index=False, sep=',')


In [20]:
# Optional: Evaluate on training set (to see how the model performed)
y_train_pred_prob = model.predict(X_train_balanced)
y_train_pred = np.argmax(y_train_pred_prob, axis=1)
mcc = matthews_corrcoef(y_train_balanced, y_train_pred)
print(f"Training MCC: {mcc}")

10/10 [==============================] - 0s 1ms/step
Training MCC: 0.0
